# Mục đích

- Đảm bảo chắc chắn là class chỉ có 1 đối tượng (instance) duy nhất, và cung cấp một điểm truy cập global đến đối tượng đó

# Sử dụng

- Khi mà app chỉ cần duy nhất một đối tượng.
- Yêu cầu:
    - Được khởi tạo lười (lazy initialization): phương pháp trì hoãn việc khởi tạo (hoặc nạp) dữ liệu cho đến khi chúng thực sự cần đến.
    - Có thể truy cập global
    
# Implement

## Cách 1: [tham khảo](https://sourcemaking.com/design_patterns/singleton/python/1), [Singletons and metaclasses](https://learning.oreilly.com/library/view/learning-python-design/9781785888038/ch02s05.html)

Trước tiên phải hiểu `metaclass`, đó là class của class, nghĩa là class sẽ là 1 thực thể của `metaclass`

`metaclass` cho phép ta tạo ra kiểu class của riêng mình, từ 1 class đã định nghĩa trước.

Chi tiết đây này, trong Python mọi thứ đều là object, ví dụ nếu `a=5` thì `type(a)` trả về `<type 'int'>`. Nghĩa là a kiểu int. Tuy nhiên `type(int)` sẽ trả về `<type 'type'>`. Nó gợi ý về sự hiện diện của `metaclass`, vì int là một lớp kiểu `type`

Định nghĩa của 1 class được quyết định bởi `metaclass`, vì vậy khi tạo 1 class, ví dụ `class A`, Python tạo nó bằng cách: `A = type(name, base, dict)`, trong đó:
- name: tên của class
- base: là base class
- dict: là biến các thuộc tính 

Bây giờ, nếu một class có một `metaclass` được định nghĩa trước, ví dụ đặt tên là `MetaKls`, lúc này Python sẽ tạo class `A =  MetaKls(name, bases, dict)`

Xem ví dụ sau về `metaclass`

In [56]:
class MyInt(type):
    def __call__(cls, *args, **kwargs):
        print("***** Hello, here is My int *****")
        print("Now, do whatever you want with these objects...")
        return type.__call__(cls, *args, **kwargs)
    
class int(metaclass=MyInt):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return "This is MyInt object"
        
i = int(1,2)
print(i)

***** Hello, here is My int *****
Now, do whatever you want with these objects...
This is MyInt object


Method `__call__` được gọi khi một đội tượng được tạo từ một class đã tồn tại. Trong đoạn code trên, khi chúng ta tạo 1 thực thể của class int thì method `__call__` của `MetaKls` được gọi, chứng tỏ `metaclass` điểu khiển quá trình khởi tạo thực thể, điều này quá toẹt vời, hế hế...

Chúng ta có thê sử dụng tính chất trên để tạo ra `Singleton` pattern

**Chú ý:** để điều khiển quá trình khởi động của một class, `metaclass` sẽ ghi đè phương thức `__new__` và `__init__`

Ta tạo một `metaclass` Singleton:

In [57]:
class SingletonMetaClass(type):
    
    def __init__(cls, name, bases, attrs, **kwargs):
        super().__init__(name, bases, attrs)
        cls.__instance = None
        # print("__init__ of SingletonMetaClass")
        
    def __call__(cls, *args, **kwargs):
        # print("__call__ of SingletonMetaClass")
        if cls.__instance is None:
            cls.__instance = super().__call__(*args, **kwargs)
        return cls.__instance

Bây giờ tạo 1 Singleton class từ thực thể trên

In [58]:
class Singleton1(metaclass=SingletonMetaClass):
    def __init__(self):
        #print("__init__ of Singleton1")
        pass

Bây giờ tạo thực thể từ class trên xem nó có là 1 không nhá

In [59]:
m1 = Singleton1()
print(m1)

m2 = Singleton1()
print(m2)


Tèn tén, kết quả in ra và kiểm tra là giống nhau, vậy nó đảm bảo tính chất là luôn luôn chỉ có duy nhất 1 thực thể của class được tạo ra rồi nhá

Còn `lazy initialization` thì sao nhỉ, trong Python có thể dùng `@property` theo link sau:

[how-can-i-delay-the-init-call-until-an-attribute-is-accessed](https://stackoverflow.com/questions/45194553/how-can-i-delay-the-init-call-until-an-attribute-is-accessed)

[python-class-member-lazy-initialization](https://stackoverflow.com/questions/15226721/python-class-member-lazy-initialization)

[lazy-properties](https://stevenloria.com/lazy-properties/)

Ví dụ bên dưới, thuộc tính `_foo` chỉ được khởi tạo khi gọi property `foo`, nếu không nó vẫn là `None`

In [60]:
class Singleton1(metaclass=SingletonMetaClass):
    def __init__(self):
        print("__init__ of Singleton1")
        self._foo = None

    @property
    def foo(self):
        if not self._foo:
            print("lazy initialization")
            self._foo =  [1,2,3]
        return self._foo
    
ms1 = Singleton1()
print(ms1._foo)
print(ms1.foo)

ms2 = Singleton1()
print(ms2._foo)

__init__ of Singleton1
None
lazy initialization
[1, 2, 3]
[1, 2, 3]


Với multi thread thì singleton trên còn đúng không nhỉ, test nhá, trước hết viết 1 hàm tạo đối tượng Singleton1 để chạy trong thread

In [61]:
def create_instance1(thread_name):
    s = Singleton1()
    print("{}: {}\n".format(thread_name, s))

Bây giờ, tạo thread xem thế nào

In [62]:
import threading
threading.Thread(target=create_instance1, args=('thread-1',)).start()
threading.Thread(target=create_instance1, args=('thread-2',)).start()
threading.Thread(target=create_instance1, args=('thread-3',)).start()
threading.Thread(target=create_instance1, args=('thread-4',)).start()

thread-1: <__main__.Singleton1 object at 0x7f55bc227e80>

thread-2: <__main__.Singleton1 object at 0x7f55bc227e80>

thread-3: <__main__.Singleton1 object at 0x7f55bc227e80>

thread-4: <__main__.Singleton1 object at 0x7f55bc227e80>



Toẹt vời, với cách này, ta có thể sử dụng cho cả multi thread :D

## Cách 2: [tham khảo](https://www.tutorialspoint.com/python_design_patterns/python_design_patterns_singleton.htm)

Hoạt động của cách này:
- dùng 1 biến private để lưu thực thể của class đó, 
- dùng 1 static method tạo thực thể, xong lưu vào biến pravite
- thực thể chỉ có thể tạo bằng static method, hoặc tạo 1 lần duy nhất bằng cách gọi tên class, tạo lần 2 bằng tên class sẽ báo lỗi


In [63]:
class Singleton2:
    __instance = None
    
    @staticmethod
    def get_instance():
        if Singleton2.__instance is None:
            Singleton2()
        return Singleton2.__instance
    
    def __init__(self):
        if Singleton2.__instance is not None:
            raise Exception("This class is singleton!")
        else:
            Singleton2.__instance = self

Bây giờ, tạo thực thể của class trên để test nhá :D

In [64]:
s = Singleton2()
print(s)
s = Singleton2.get_instance()
print(s)
s = Singleton2.get_instance()
print(s)

Tèn tén, giống nhau rồi nhá, bây giờ tạo một thực thể mới từ tên class không qua static method xem thế nào nhá

In [65]:
s = Singleton2()

Exception: This class is singleton!

Há há, lỗi rồi...vậy là đạt yêu cầu

Tham khảo phần **Lazy initialization** ở cách 1

Tét với multi thread nhá:

In [ ]:
def create_instance2(thread_name):
    s = Singleton2.get_instance()
    print("{}: {}\n".format(thread_name, s))
    
threading.Thread(target=create_instance2, args=('thread-1',)).start()
threading.Thread(target=create_instance2, args=('thread-2',)).start()
threading.Thread(target=create_instance2, args=('thread-3',)).start()
threading.Thread(target=create_instance2, args=('thread-4',)).start()

Ngon, cũng chạy được với multi thread. :D

## Cách 3: [tham khảo branch forked](https://github.com/PhungXuanAnh/python-patterns/blob/forked/patterns/creational/borg.py)

Cách này sử dụng một tính chất của Python, đó là các thuộc tính của 1 thực thể thì được lưu trong 1 dictionary tên là `__dict__`. Vậy là bây giờ đơn giản là làm cho tất cả các thực thể có cùng `__dict__`, tức là cùng thuộc tính thì sẽ đáp ứng yêu cầu có duy nhất 1 thực thể được tạo ra, phải không nhể 

Trong cách này:
- Biến `__shared_state` sẽ dùng để lưu thuộc tính share giữa các thực thể
- Khi tạo thực thể mới thì gán `__shared_state` cho `__dict__`


# [Kịch bản thực thực tế 1](https://learning.oreilly.com/library/view/learning-python-design/9781785888038/ch02s06.html)

Ví dụ với 1 web app có nhiều hoạt động đọc ghi database, app này được chia thành nhiều service thực hiện các hoạt động trên database. Lúc này nguồn tài nguyên chia sẻ giữa các service chính là database. Vì vậy để app hoạt động tốt, những điểm sau cần được quan tâm:

- Tính nhất quán trong các hoạt động với database - một hành động không được xung đột với các hành động khác
- Sử dụng bộ nhớ và cpu phải được tối ưu cho việc sử lý nhiều hành động trên database

Code bên dưới sẽ thực hiện ví dụ này:

In [66]:
import sqlite3
class MetaSingleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, \
                cls).__call__(*args, **kwargs)
        return cls._instances[cls]

class Database(metaclass=MetaSingleton):
  connection = None
  def connect(self):
    if self.connection is None:
        self.connection = sqlite3.connect("db.sqlite3")
        self.cursorobj = self.connection.cursor()
    return self.cursorobj

db1 = Database().connect()
db2 = Database().connect()

print ("Database Objects DB1", db1)
print ("Database Objects DB2", db2)

Database Objects DB1 <sqlite3.Cursor object at 0x7f55bc2327a0>
Database Objects DB2 <sqlite3.Cursor object at 0x7f55bc2327a0>


Trong đoạn code trên, những điểm sau được đề cập:

- Tạo metaclass tên là MetaSingleton, phương thức __call__ được dùng để tạo Singleton
- Class Database được mô tả bởi MetaSingleton, vì thế nó sẽ chỉ tạo 1 đối tượng
- Khi web app muốn thực hiện các hành động với DB, nó khởi tạo đối tượng của class Database nhiều lần, nhưng chỉ 1 đối tượng duy nhất được tạo, lúc này các cuộc gọi đến DB là đồng bộ. Hơn nữa, nó cũng sử dụng ít tài nguyền hệ thống hơn
    
**Chú ý**: với tình huống có nhiều web app, nhưng chỉ có 1 DB, lúc này Singleton không còn phù hợp nữa, vì mỗi web app sẽ tạo cho nó 1 đối tượng Singleton, dẫn đến các hoạt động với DB trở thành bất đồng bộ và sử dụng nhiều tài nguyên hơn. Trong những trường hợp như vậy thì cách tốt hơn là sử dụng `connection pool`


# [Kịch bản thực thực tế 2](https://learning.oreilly.com/library/view/learning-python-design/9781785888038/ch02s07.html)

Một kịch bản khác, khi chúng ta thực hiện 1 dịch vụ kiểm tra xem máy chủ có làm việc đúng hay không (server health check) ví dụ như Nagios. Tạo 1 class Singleton tên là HealthCheck. Dịch vụ này sẽ kiểm tra 1 danh sách các server bằng `health check` tương ứng của các server đó. Nếu 1 server bị gỡ bỏ thì dịch vụ này phải biết và gỡ bỏ nó khỏi cấu hình server.

Trong đoạn code sau, `hc1` và `hc2` là đối tượng của một class Singleton

Server được thêm vào bằng phương thức `addServer()`. Trong lần chạy đầu tiên, dịch vụ sẽ kiểm tra với danh sách các server được thêm vào này. Sau đó phương thức `changeServer()` sẽ  gỡ bỏ server cuối cùng và thêm vào 1 server khác. Trong lần chạy thứ 2, dịch vụ sẽ kiểm tra với danh sách servers mới này.

In [67]:
class HealthCheck:
    _instance = None
    def __new__(cls, *args, **kwargs):
        if not HealthCheck._instance:
            HealthCheck._instance = super(HealthCheck, \
                cls).__new__(cls, *args, **kwargs)
        return HealthCheck._instance
    def __init__(self):
        self._servers = []
    def addServer(self):
        self._servers.append("Server 1")
        self._servers.append("Server 2")
        self._servers.append("Server 3")
        self._servers.append("Server 4")
    def changeServer(self):
        self._servers.pop()
        self._servers.append("Server 5")

hc1 = HealthCheck()
hc2 = HealthCheck()

hc1.addServer()
print("Schedule health check for servers (1)..")
for i in range(4):
    print("Checking ", hc1._servers[i])

hc2.changeServer()
print("Schedule health check for servers (2)..")
for i in range(4):
    print("Checking ", hc2._servers[i])

Schedule health check for servers (1)..
Checking  Server 1
Checking  Server 2
Checking  Server 3
Checking  Server 4
Schedule health check for servers (2)..
Checking  Server 1
Checking  Server 2
Checking  Server 3
Checking  Server 5
